In [1]:
import json
from pymongo import MongoClient
from googletrans import Translator
from bson.objectid import ObjectId
import tqdm
import re
from nltk import ngrams
import Levenshtein
import pickle
import os
from gensim.test.utils import lee_corpus_list
from gensim.models import Word2Vec, Doc2Vec
import json
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
f = open('ontology.json', 'r')
g_onto = json.loads(f.read())

def get_all_skill_from_onto(d_in, skill, exception, level):
    for k in d_in:
        if type(d_in[k]) == dict:
            level+=1
            skill = get_all_skill_from_onto(d_in[k], skill, exception, level)
            level-=1
            if k not in exception and level >=2:
                skill.add(k)
                
    return skill
len(get_all_skill_from_onto(g_onto, set(), ['database', ''], 0))

11731

In [3]:
f = open('ontology.json', 'r')
g_onto = json.loads(f.read())

def fix_g_onto(d_in, new_onto, exception, level):
    for k in d_in:
        if type(d_in[k]) == dict:
            level+=1
            new_onto = fix_g_onto(d_in[k], new_onto, exception, level)
            level-=1
            
            temp=[]
            for i in d_in[k].keys():
                if i in exception and level >=2:
                    temp.append(i)
            
            for i in temp:
                d_in[k].pop(i, None)
                
    return d_in
new_onto = fix_g_onto(g_onto, {}, ['smartphones', 'smart phone'], 0)

In [4]:
f = open('ontology.json', 'r')
g_onto = json.loads(f.read())

def fix_g_onto(d_in, new_onto, exception, level):
    for k in d_in:
        if type(d_in[k]) == dict:
            level+=1
            fix_g_onto(d_in[k], new_onto, exception, level)
            level-=1
            
            temp=[]
            for i in d_in[k].keys():
                if i in exception and level >=2:
                    print(i)
                
    return
fix_g_onto(new_onto, {}, ['smartphones', 'smart phone'], 0)

In [5]:
client = MongoClient(host='103.252.1.144', username='admin', password = 'CIST2o20')
cb = client['human_resources']['career_builder']
cv = cb.find({'eng_experience':{'$exists':True}, 'fulltext':{'$regex':'.*developer.*'}})
jpt = client['human_resources']['job_posting_topcv']
jd = jpt.find({'eng_job_description':{'$regex':'.*developer.*'}})

In [657]:
class CSO_classifier():
    def __init__(self, path='ontology.json'):
        f = open(path, 'r')
        self.g_onto = json.loads(f.read())
        self.g_onto = self.fix_g_onto(g_onto, {}, ['smartphones', 'smart phone'], 0)
        self.skill = self.get_all_skill_from_onto(self.g_onto, set(),['database', 'knowledge', 'research', 'drawing', 'engineer', 'desktop'], 0)


    def get_all_skill_from_onto(self, d_in, skill, exception, level):
        for k in d_in:
            if type(d_in[k]) == dict:
                level+=1
                skill = self.get_all_skill_from_onto(d_in[k], skill, exception, level)
                level-=1
                if k not in exception and level >=2:
                    skill.add(k)
        return skill

    def fix_g_onto(self, d_in, new_onto, exception, level):
        for k in d_in:
            if type(d_in[k]) == dict:
                level+=1
                new_onto = self.fix_g_onto(d_in[k], new_onto, exception, level)
                level-=1
                
                temp=[]
                for i in d_in[k].keys():
                    if i in exception and level >=2:
                        temp.append(i)
                
                for i in temp:
                    d_in[k].pop(i, None)
                    
        return d_in

    def get_skill_by_concept_similarity(self, sentence):
        sentence = re.sub(r"[-()\"/@;:<>{}`=~|.!?,]", " ", sentence).lower()

        uni_gram = ngrams(sentence.split(), 1)
        bi_gram = ngrams(sentence.split(), 2)
        tri_gram = ngrams(sentence.split(), 3)

        uni_gram = [i[0] for i in list(uni_gram)]
        bi_gram = ['{} {}'.format(i[0],i[1]) for i in list(bi_gram)]
        tri_gram = ['{} {} {}'.format(i[0],i[1], i[2]) for i in list(tri_gram)]
        
        n_gram = uni_gram + bi_gram + tri_gram

        skill_find_out = set()
        
        for i in n_gram:
            for o_k in self.skill:
                if Levenshtein.ratio(i, o_k) > 0.94:
                    skill_find_out.add(o_k)

        return skill_find_out

    
    def build_graph(self, onto, skill, level, skill_same_level, graph):
        for i in onto:
            level+=1
            if type(onto[i]) == dict:
                level, skill_same_level, graph = self.build_graph(onto[i], skill, level, skill_same_level, graph)
    
                pop_ = []
                for j in graph.keys():
                    if j in onto[i].keys():
                        pop_.append(j)
                for j in pop_:
                    if j!= '':
                        tmp = graph[j]
                        graph.pop(j, None)
                        if i not in graph:
                            graph[i]={}
                        graph[i][j] = tmp

                if i in skill:
                    if level not in skill_same_level:
                        skill_same_level[level]=set()
                    skill_same_level[level].add(i)

                if level+1 in skill_same_level:
                    if len(skill_same_level[level+1]) > 0:
                        way_1 = False
                        if i not in graph:
                            graph[i] = list(skill_same_level[level+1])
                        else:
                            for j in skill_same_level[level+1]:
                                if j in graph[i]:
                                    way_1 = True
                            if not way_1 and len(graph[i])>0 and type(graph[i])==dict:
                                for k in skill_same_level[level+1]:
                                    graph[i][k]=[k]
                        
                        skill_same_level[level+1]=set()
                
                level-=1
        return level, skill_same_level, graph
    
    def count_skill_from_graph(self, graph, skill, level, new_graph, cnt=None):
        if(type(graph) == dict):
            for i in graph.keys():
                level+=1
                if cnt!=None:
                    if level==3:
                        cnt.append({i:0})
                
                if type(graph[i]) == dict:
                    skill, cnt, level, new_graph = self.count_skill_from_graph(graph[i], skill, level, new_graph, cnt) 
                    level-=1  
                else:
                    for j in graph[i]:
                        skill.add(j)
                        if cnt!=None and len(cnt)>0:
                            cnt[-1][list(cnt[-1].keys())[0]] += 1
                    level-=1            

        return skill, cnt, level, new_graph

    def get_skill_from_graph(self, graph, skill, level, new_graph, cnt=None):
        if type(graph) == dict:
            for i in graph.keys():
                level+=1
                if cnt!=None:
                    if level==3:
                        cnt.append({i:0})
                
                if type(graph[i]) == dict:
                    skill, cnt, level, new_graph = self.get_skill_from_graph(graph[i], skill, level, new_graph, cnt) 
                    level-=1  
                else:
                    for j in graph[i]:
                        skill.add(j)
                        if cnt!=None and len(cnt)>0:
                            cnt[-1][list(cnt[-1].keys())[0]] += 1
                    level-=1            

            return skill, cnt, level, new_graph
        else:
            return graph

    def build_skill_graph(self, all_skill):
        new_graph = {}
        graph =  self.build_graph(g_onto, all_skill, 0, {}, {})[2]
        count_domain = self.count_skill_from_graph(graph, set(), 0, {}, [])[1]
        not_cancel = True
        
        while len(all_skill)>0 and not_cancel:
            
            max_domain = {'domain': '', 'count': 0}
            for i in count_domain:
                if i[list(i.keys())[0]] > max_domain['count']:
                    max_domain['domain'] = list(i.keys())[0]
                    max_domain['count'] = i[list(i.keys())[0]]

            for i in graph:
                for k in graph[i]:
                    for j in graph[i][k]:
                        if j == max_domain['domain']:
                            sub_skill = self.get_skill_from_graph(graph[i][k][j], set(), 0, {})[0]
                            if i not in new_graph:
                                new_graph[i] = {}
                            if k not in new_graph[i]:
                                new_graph[i][k] = {}
                            if j not in new_graph[i][k]:
                                new_graph[i][k][j] = {}
                            
                            new_graph[i][k][j] = graph[i][k][j]

            if type(sub_skill) == str:
                all_skill=all_skill-set([sub_skill])
            else:
                all_skill=all_skill-sub_skill

            graph = self.build_graph(g_onto, all_skill, 0, {}, {})[2]

            count_domain = self.count_skill_from_graph(graph, set(), 0, {}, [])[1]
            
            if len(count_domain)==0:
                not_cancel = False
        return new_graph

                                                                                                                    

In [658]:
ontology = {'A':[], 'JD': [], 'R': []}

c = {'Profile': None, 'Work_history': None, 'Education': None, 'Course': None, 'Skills': {}, 'Pos': None, 'Others': None}
d = {'Information': None, 'Descriptions': None, 'Required': None, 'Skills': {}}

cso_classifier = CSO_classifier()

In [649]:
skills = cso_classifier.get_skill_by_concept_similarity(' '.join(cv[0]['eng_experience']+cv[0]['eng_skills']))


In [659]:
cso_classifier.build_skill_graph(skills)

{'general': {'computer science': {'software engineering': {'software design': {'software architecture': {'software development': ['software team'],
      'software development process': ['software team'],
      'software development life cycle': ['software team'],
      'software development project': ['software team'],
      'software developer': ['software developer']},
     'software product': {'software development': ['software team'],
      'software development process': ['software team'],
      'software development life cycle': ['software team'],
      'software development project': ['software team']},
     'software project': {'software development': ['software team'],
      'software development process': ['software team'],
      'software development life cycle': ['software team'],
      'software development project': ['software team']},
     'software process': {'software development': ['software team'],
      'software development process': ['software team'],
      'soft